# Prepairing chesscom data to use

## Import and functions

In [14]:
# coding: utf-8

# our all
import numpy as np
import pandas as pd

# usefull pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for API working and current time
import requests
import datetime

# chess pgn-reading tool
from pgn_parser import pgn, parser

# multistreaming
import threading

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

In [2]:
# API, read and write finctions

# convert unixtime to time:
# fr_unixtime(1565211491) -> '2019-08-07 20:58:11'
def fr_unixtime(ts):
    from datetime import datetime
    return datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d %H:%M:%S')

# convert integer month number to string API format:
# 1 -> '01' 
# 9 -> '09' 
# 12 > '12'
# 112 -> -1
def get_number(x):
    if x>=10 and x<100:
        return str(x)
    elif x<10:
        return '0'+str(x)
    else:
        return -1

# get json form site and return string
# get_bch('eric', '2014', '01')
def get_bch(player, year, month):
    import requests
    response = requests.get('https://api.chess.com/pub/player/'+player+'/games/'+year+'/'+month)
    st=response.text
    if response.status_code !=200:
        print(response.status_code)
        print(st[0:3000])
    return st

# write string (json) to folder
# write_bch('eric', '2014', '01') -> 'source/eric_2014_01_2019-08-07 20:58:11.json'
# TO DO: fix attemps cikle
def write_bch(player, year, month):
    # get current time to log chenges in filenames
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # we give 3 attemps to get data
    for cs in range(3):
        try:
            st=get_bch(player, year, month)
            file = open('source/'+player+'_'+year+'_'+month+'_'+now+'.json','w') 
            file.write(st)
            file.close()
        except Exception:
            # wait 0,1 or 2 seconds after next attempt
            time.sleep(cs)
            error_st=response.status_code+st[0:3000]
            file = open('source/'+player+'_'+year+'_'+month+'_'+now+'.json','w') 
            file.write( error_st)
            file.close()

# get players list by countries
def get_county_players_list(country):
    import requests
    response = requests.get('https://api.chess.com/pub/country/'+country+'/players')
    st=response.text
    if response.status_code !=200:
        print(response.status_code)
        print(st[0:3000])
        
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
 
    try:
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write(st)
        file.close()
    except Exception:
        error_st=response.status_code+st[0:3000]
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write( error_st)
        file.close()
# read data fromjson file and prepair to future work
def get_data_from_file(path):
    total_info=pd.read_json(path)
    
    # number of games
    num=len(total_info)

    game_stat=pd.DataFrame()
    for i in range(num):
        b=pd.io.json.json_normalize(total_info.values[i])
        game_stat=pd.concat([game_stat, b])
        
    return game_stat
# df=get_data_from_file('source/rosolimo_2018_04_2019-08-04 22:21:13.json')


In [3]:
# specific data preparation
# chess specific functions

# retunr classoc Elo propabilities
# elo_prob(2882, 2722) -> 0.7152 (72% chanses Carlsen (2882) to beat Wan Hao (2722))
def elo_prob(rw, rb):
    return 1/(1+np.power(10, (rb-rw)/400))

# working with pgn
# pip install pgn-parser
def get_pgn(text):
    from pgn_parser import pgn, parser
    game = parser.parse(text, actions=pgn.Actions())
    
    try:
        score=game.tag_pairs['Result']
    except:
        score='Unknown'  
    try:
        date=game.tag_pairs['Date']
    except:
        date='Unknown'
    try:
        time=game.tag_pairs['UTCTime']
    except:
        time='Unknown'
    try:
        eco=game.tag_pairs['ECO']
    except:
        eco='Unknown'
    try:
        ecourl=game.tag_pairs['ECOUrl']
    except:
        ecourl='Unknown'
    return {'score':score, 
            'date': date, 
            'time':time, 
            'ECO':eco, 
            'ECO_url':ecourl}

# get player games in month with preparation
def get_pl_stat(game_stat):    
    # chess result dictionary
    res_dict={
    'win': 1.0,
    'checkmated': 0.0,
    'agreed': 0.5,
    'repetition': 0.5,
    'timeout': 0.0,
    'resigned': 0.0,
    'stalemate': 0.5,
    'lose': 0.0,
    'insufficient': 0.5,
    '50move': 0.5,
    'abandoned': 0.0,
    'kingofthehill': 0.5,
    'threecheck': 0.5,
    'timevsinsufficient': 0.5,
    'bughousepartnerlose': 0.0
    }

    # number of games
    num=len(game_stat)
    
    # cut long and unusefull columns
    game_stat=game_stat[['rules', 'time_class', 'time_control', 'rated',
        'white.@id', 'white.rating', 'white.result', 'white.username',
         'black.@id', 'black.rating', 'black.result', 'black.username',
         'end_time', 'pgn', 
         'url']]
    # score 0.0, 0.5 or 1.0
    game_stat['white.score']=game_stat['white.result'].map(res_dict)
    game_stat['black.score']=game_stat['black.result'].map(res_dict)
    
    # pgn - long string
    game_stat['pgn']=game_stat['pgn'].apply(get_pgn)
    # result - string like '1-0'
    game_stat['result']=game_stat['pgn'].apply(lambda x: x['score'])
    
    game_stat['date']=game_stat['pgn'].apply(lambda x: x['date'])
    game_stat['time']=game_stat['pgn'].apply(lambda x: x['time'])
    
    # ECO - string like 'B10'
    game_stat['eco']=game_stat['pgn'].apply(lambda x: x['ECO'])
    # ECOurl - url like 'https://www.chess.com/openings/B10-Caro-Kann-Defense-2.Nf3-d5'
    game_stat['eco_url']=game_stat['pgn'].apply(lambda x: x['ECO_url'])
    
    # propability from classic formula
    game_stat['white_elo_forecast']=game_stat[['white.rating', 'black.rating']].apply(lambda x: elo_prob(*x), axis=1)
    game_stat['black_elo_forecast']=game_stat[['black.rating', 'white.rating']].apply(lambda x: elo_prob(*x), axis=1)
    
    # after it np.sum(df['game']) means number of games
    game_stat['game']=1
    game_stat['date'] = game_stat['date'].astype('datetime64[ns]')
    # period - stirng like '2019-08'
    game_stat['period']=game_stat['date'].dt.year.astype('str')+'-'+game_stat['date'].dt.month.astype('str')
        
    game_stat.columns=['rules', 'time_class', 'time_control', 'rated', 'white_url',
       'white.rating', 'white.result', 'white.username', 'black_url',
       'black.rating', 'black.result', 'black.username', 'end_time', 'pgn', 'game_url',
       'white.score', 'black.score', 'result', 'date', 'time', 'eco',
       'eco_url', 'white_elo_forecast', 'black_elo_forecast',
       'game', 'period']
    game_stat=game_stat[[
        'rules', 'time_class', 'time_control', 'rated', 'game',
        'result', 'date', 'time', 'period',
        'white.username', 'black.username',
        'white.rating', 'black.rating', 
        'white.result', 'black.result', 
        'white.score', 'black.score', 
        'white_elo_forecast', 'black_elo_forecast',
        'eco', 'eco_url', 
        'game_url', 'white_url', 'black_url'
    ]]
    return game_stat

# functions for predictions
# find col with target user (white or black)
def find_col(target_user, col1, wh_val, bl_val):
    if col1==target_user:
        return wh_val
    else:
        return bl_val
# reverse fun of find_col    
def find_opp(target_user, col1, wh_val, bl_val):
    if col1!=target_user:
        return wh_val
    else:
        return bl_val

# data for predict current user
def learn_prepair(df, target_user):
    df['target_user']=target_user
    df['score']=df[['target_user', 'white.username', 'white.score', 'black.score']].apply(lambda x: find_col(*x), axis=1)
    df['rating']=df[['target_user','white.username', 'white.rating', 'black.rating']].apply(lambda x: find_col(*x), axis=1)
    df['opponent_rating']=df[['target_user', 'white.username', 'white.rating', 'black.rating']].apply(lambda x: find_opp(*x), axis=1)
    df['base_elo_forec']=df[['target_user', 'white.username', 'white_elo_forecast', 'black_elo_forecast']].apply(lambda x: find_col(*x), axis=1)
    df['color']=np.where(df['white.username']==target_user, 'w', 'b')
    
    return df

In [4]:
# start getting with threading magic
def user_parse(user_list, year_range):   
    import threading
    # status counter
    i=0
    for user in user_list:
        for year in year_range:
            for month in range(1,13):
                t = threading.Thread(target=write_bch, args=(user, str(year), get_number(month)))
                t.start()
                i=i+1
    print(i)
    
# start getting with threading magic
def file_parse(user_list, year_range):   
    import threading
    # status counter
    i=0
    for user in user_list:
        for year in year_range:
            for month in range(1,13):
                t = threading.Thread(target=write_bch, args=(user, str(year), get_number(month)))
                t.start()
                i=i+1
    print(i)

In [15]:
# read all json in folder and concate DataFrame
def read_files(path, is_normalise):
    # reading
    l=[]
    cntr=0
    import os
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".json"):
                 l.append(os.path.join(root, file))
            cntr=cntr+1
    print('Total', cntr, 'files founded')

    # merging
    i=0
    df=pd.DataFrame()
    for link in l:
        try:
            if is_normalise==1:
                dfb=get_data_from_file(link)
                dfb=get_pl_stat(dfb)
            else:
                dfb=pd.read_json(link)
        except:
            dfb=pd.DataFrame()
        df=pd.concat([df, dfb])
        
    return df

## Start working

In [16]:
# collect players by countries
countries_list=['RU']
get_county_players_list(countries_list[0])

#user_list=read_files('player_lists/', 0)['players']
user_list=['rosolimo']

In [7]:
# write to files 
user_parse(user_list, range(2001, 2020))

Exception in thread Thread-96:
Traceback (most recent call last):
  File "<ipython-input-2-22ffabb77126>", line 44, in write_bch
    st=get_bch(player, year, month)
  File "<ipython-input-2-22ffabb77126>", line 26, in get_bch
    response = requests.get('https://api.chess.com/pub/player/'+player+'/games/'+year+'/'+month)
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/requests/api.py", line 75, in get
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/requests/api.py", line 60, in request
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/requests/sessions.py", line 533, in request
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/requests/sessions.py", line 646, in send
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/requests/adapters.py", line 449, in send
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/urllib3/connectio

Exception in thread Thread-94:
Traceback (most recent call last):
  File "<ipython-input-2-22ffabb77126>", line 44, in write_bch
    st=get_bch(player, year, month)
  File "<ipython-input-2-22ffabb77126>", line 26, in get_bch
    response = requests.get('https://api.chess.com/pub/player/'+player+'/games/'+year+'/'+month)
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/requests/api.py", line 75, in get
    return request('get', url, params=params, **kwargs)
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/requests/api.py", line 60, in request
    return session.request(method=method, url=url, **kwargs)
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/requests/sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/requests/sessions.py", line 646, in send
    r = adapter.send(request, **kwargs)
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/requests/adapters.py", lin

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Exception in thread Thread-29:
Traceback (most recent call last):
  File "<ipython-input-2-22ffabb77126>", line 45, in write_bch
    file = open('source/'+player+'_'+year+'_'+month+'_'+now+'.json','w')
OSError: [Errno 24] Too many open files: 'source/rosolimo_2003_02_2019-08-09 15:03:09.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/roman/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/roman/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-22ffabb77126>", line 50, in write_bch
    time.sleep(cs)
NameError: name 'time' is not defined
Unhandled exception in thread started by Exception in thread Thread-17:
Traceback (most recent call last):
  File "<ipython-input-2-22ffabb77126>",

228


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 61, in create_connection
    sock = socket.socket(af, socktype, proto)
  File "/Users/roman/anaconda3/lib/python3.7/socket.py", line 151, in __init__
    _socket.socket.__init__(self, family, type, proto, fileno)
OSError: [Errno 24] Too many open files

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/Users/roman/anaconda3/lib/python3.7/site-package

Exception in thread Thread-23:
Traceback (most recent call last):
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 57, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Users/roman/anaconda3/lib/python3.7/socket.py", line 748, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 343, in _make_reques

Traceback (most recent call last):
  File "/Users/roman/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ZMQError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
429429  File "/Users/roman/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)

  File "/Users/roman/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)


  File "/Users/roman/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 347, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))

429

  File "/Users/roman/anaconda3/lib/python3.7/inspect.py", line 1502, in getinnerframes
   

In [17]:
df=read_files('source/', is_normalise=1)

Total 102 files founded


In [18]:
# try sample
df.sample(9)

,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url
0,chess,blitz,300,True,1,0-1,2014-06-14,18:56:40,2014-6,Rosolimo,gregsecure1,1511,1509,checkmated,win,0.0,1.0,0.502878,0.497122,A41,https://www.chess.com/openings/A41-Queens-Pawn-Opening-Anglo-Slav-Opening,https://www.chess.com/live/game/834472224,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/gregsecure1
0,chess,blitz,300,True,1,1-0,2014-06-23,19:56:10,2014-6,Rosolimo,JacobSteinbauer,1523,1529,win,resigned,1.0,0.0,0.491366,0.508634,D10,https://www.chess.com/openings/D10-Slav-Defense-3.Nc3-dxc4,https://www.chess.com/live/game/842298511,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/jacobsteinbauer
0,chess,blitz,300,True,1,1-0,2016-09-16,19:19:43,2016-9,Rosolimo,Leonardon,1679,1727,win,timeout,1.0,0.0,0.431359,0.568641,D02,https://www.chess.com/openings/D02-Queens-Gambit-Declined-Baltic-Queen-Attack,https://www.chess.com/live/game/1732001259,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/leonardon
0,chess,blitz,300,True,1,1-0,2013-10-02,10:18:18,2013-10,Rosolimo,rajeevkuda,1479,1439,win,resigned,1.0,0.0,0.557312,0.442688,E92,https://www.chess.com/openings/E92-Kings-Indian-Defense-Petrosian-Variation,https://www.chess.com/live/game/613382853,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/rajeevkuda
0,chess,blitz,300+3,False,1,1-0,2016-09-07,10:59:53,2016-9,P_Rekhtman,Rosolimo,2089,1676,win,resigned,1.0,0.0,0.915089,0.084911,D02,https://www.chess.com/openings/D02-Queens-Pawn-Opening-Symmetrical-Variation,https://www.chess.com/live/game/1719966927,https://api.chess.com/pub/player/p_rekhtman,https://api.chess.com/pub/player/rosolimo
0,chess,blitz,300,True,1,1-0,2014-11-29,15:11:37,2014-11,twobottleslater,Rosolimo,1627,1559,win,resigned,1.0,0.0,0.596629,0.403371,D03,https://www.chess.com/openings/D03-Queens-Pawn-Opening-Torre-Gruenfeld-Variation,https://www.chess.com/live/game/983917948,https://api.chess.com/pub/player/twobottleslater,https://api.chess.com/pub/player/rosolimo
0,chess,blitz,300,True,1,1-0,2015-12-13,10:18:40,2015-12,Rosolimo,chitasean,1547,1525,win,resigned,1.0,0.0,0.531618,0.468382,A40,https://www.chess.com/openings/A40-English-Defense-2.c4-Bb7-3.Nc3,https://www.chess.com/live/game/1383451290,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/chitasean
0,chess,blitz,300,True,1,1-0,2014-03-17,14:58:27,2014-3,Rosolimo,salvadoroflores,1491,1554,win,timeout,1.0,0.0,0.410316,0.589684,D10,https://www.chess.com/openings/D10-Slav-Defense,https://www.chess.com/live/game/754209708,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/salvadoroflores
0,chess,blitz,300,True,1,0-1,2014-03-04,10:58:47,2014-3,Rosolimo,Keithpitt,1520,1516,checkmated,win,0.0,1.0,0.505756,0.494244,A85,https://www.chess.com/openings/A85-Dutch-Defense-Queens-Knight-Variation,https://www.chess.com/live/game/742663979,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/keithpitt


In [19]:
# focus in one user to predict
lrn=learn_prepair(df, 'Rosolimo')

In [20]:
lrn.sample(9)

,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url,target_user,score,rating,opponent_rating,base_elo_forec,color
0,chess,blitz,300,True,1,1-0,2015-08-06,09:20:06,2015-8,olive-41,Rosolimo,1547,1572,win,resigned,1.0,0.0,0.464084,0.535916,B13,https://www.chess.com/openings/B13-Caro-Kann-Defense-Exchange-Variation-3...cxd5-4.Nf3,https://www.chess.com/live/game/1238819457,https://api.chess.com/pub/player/olive-41,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.0,1572,1547,0.535916,b
0,chess,blitz,300,True,1,0-1,2014-03-30,20:18:20,2014-3,manigoldoelcid,Rosolimo,1489,1539,checkmated,win,0.0,1.0,0.428537,0.571463,B10,https://www.chess.com/openings/B10-Caro-Kann-Defense-Hillbilly-Attack-2...d5,https://www.chess.com/live/game/766001737,https://api.chess.com/pub/player/manigoldoelcid,https://api.chess.com/pub/player/rosolimo,Rosolimo,1.0,1539,1489,0.571463,b
0,chess,blitz,300,True,1,1-0,2014-09-12,19:25:28,2014-9,Rosolimo,NorthLite,1524,1535,win,timeout,1.0,0.0,0.484175,0.515825,D10,https://www.chess.com/openings/D10-Slav-Defense-3.Nc3,https://www.chess.com/live/game/912740309,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/northlite,Rosolimo,1.0,1524,1535,0.484175,w
0,chess,blitz,300,True,1,0-1,2014-03-07,20:31:31,2014-3,Rosolimo,Axelj48,1485,1558,resigned,win,0.0,1.0,0.396464,0.603536,A85,https://www.chess.com/openings/A85-Dutch-Defense-Queens-Knight-Variation,https://www.chess.com/live/game/745724221,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/axelj48,Rosolimo,0.0,1485,1558,0.396464,w
0,chess,blitz,300,True,1,1-0,2014-06-20,21:40:09,2014-6,Rosolimo,NimrodToister,1541,1483,win,checkmated,1.0,0.0,0.582702,0.417298,A20,https://www.chess.com/openings/A20-English-Opening-Kings-English-Variation-2.e3-d6-3.Nc3,https://www.chess.com/live/game/839762918,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/nimrodtoister,Rosolimo,1.0,1541,1483,0.582702,w
0,chess,blitz,180+2,True,1,1-0,2016-09-17,19:18:07,2016-9,offtherook,Rosolimo,1581,1650,win,resigned,1.0,0.0,0.401986,0.598014,B18,https://www.chess.com/openings/B18-Caro-Kann-Defense-Classical-Variation-5.Ng3-Bg6-6.h4-h6-7.h5,https://www.chess.com/live/game/1733299269,https://api.chess.com/pub/player/offtherook,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.0,1650,1581,0.598014,b
0,chess,blitz,300,True,1,1-0,2014-03-09,20:32:49,2014-3,NeilBerm,Rosolimo,1518,1500,win,checkmated,1.0,0.0,0.525881,0.474119,B10,https://www.chess.com/openings/B10-Caro-Kann-Defense-2.Nf3-d5,https://www.chess.com/live/game/747513601,https://api.chess.com/pub/player/neilberm,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.0,1500,1518,0.474119,b
0,chess,blitz,300,True,1,0-1,2014-06-21,17:51:42,2014-6,Joshua_Ray,Rosolimo,1508,1515,resigned,win,0.0,1.0,0.489928,0.510072,B10,https://www.chess.com/openings/B10-Caro-Kann-Defense-Hillbilly-Attack-2...d5,https://www.chess.com/live/game/840467948,https://api.chess.com/pub/player/joshua_ray,https://api.chess.com/pub/player/rosolimo,Rosolimo,1.0,1515,1508,0.510072,b
0,chess,blitz,300,True,1,1-0,2014-03-08,16:57:44,2014-3,Rosolimo,xhemal,1497,1516,win,checkmated,1.0,0.0,0.472684,0.527316,D38,https://www.chess.com/openings/D38-Queens-Gambit-Declined-Ragozin-Defense-5.Bg5,https://www.chess.com/live/game/746450527,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/xhemal,Rosolimo,1.0,1497,1516,0.472684,w
